# Voronoi analysis with OVITO

This notebook mirrors `voronoi_canon.ipynb` but uses OVITO's `VoronoiAnalysisModifier` via `voronoi_ovito_utils.py`.

- Build Voronoi graphs (full and metals-only)
- Summarize graph properties and clusters
- Compute coordination distributions

Note: Ensure OVITO and ASE are installed in this environment.


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from ovito.io import import_file

from voronoi_ovito_utils import (
    build_voronoi_graph_from_pipeline,
    build_voronoi_graph_metals_only_from_pipeline,
    analyze_graph_properties,
    analyze_voronoi_clusters,  # atoms-based version (kept for reference)
    analyze_voronoi_coordination_from_pipeline,
    analyze_temporal_graph_properties_from_pipeline,
    plot_temporal_graph_properties,
)



: 

In [ ]:
# Configure your input structure/trajectory file
input_path = '/pscratch/sd/p/pvashi/irp/irp_mace_l_2/irp/density/NaCl-PuCl3/x0.67/T1300K/dump.lammpstrj'  # TODO: Set path

# Load with OVITO pipeline
aae = import_file(input_path, multiple_frames=True)
print(f"Loaded trajectory with {aae.source.num_frames} frames")
frame = 0  # choose frame index

# Compute once to confirm cell/PBC and show basic info
data0 = aae.compute(frame)
print('Cell matrix:')
print(np.array(data0.cell.matrix))
print('PBC:', tuple(bool(x) for x in data0.cell.pbc))



In [ ]:
# Full-system Voronoi graph (OVITO pipeline)
G = build_voronoi_graph_from_pipeline(aae, frame=frame, min_area=0.0)
props = analyze_graph_properties(G)
print('Graph properties (full):', props)

# Optional: summarize Voronoi edge network
from voronoi_ovito_utils import summarize_voronoi_edge_network
pair_counts = summarize_voronoi_edge_network(G, plot=True)
pair_counts


In [ ]:
# Metals-only Voronoi graph and clusters (OVITO pipeline)
metal_species = ['Pu', 'Na']  # Adjust as needed
Gm = build_voronoi_graph_metals_only_from_pipeline(aae, frame=frame, min_area=0.0, metal_species=metal_species)
print('Metals-only graph properties:', analyze_graph_properties(Gm))

# Coordination distributions over selected frames (OVITO pipeline)
frames = [frame]
coord = analyze_voronoi_coordination_from_pipeline(aae, frames=frames, at_list=None, min_area=0.0)
central = metal_species[0] if metal_species else list(coord.keys())[0]
print(f"Coordination counts for central species '{central}':")
for neighbor_sp, vals in coord.get(central, {}).items():
    if vals:
        print(neighbor_sp, '-> mean:', float(np.mean(vals)), 'std:', float(np.std(vals)))



In [ ]:
# (Optional) Temporal analysis over selected frames with OVITO pipeline
# frames = list(range(0, min(aae.source.num_frames, 50), 5))
# temporal = analyze_temporal_graph_properties_from_pipeline(aae, frames=frames, min_area=0.0)
# plot_temporal_graph_properties(temporal)

print('Notebook scaffold ready. Set input_path and run cells.')



In [ ]:
# Match plots from voronoi_canon
from voronoi_ovito_utils import (
    plot_coordination_histograms,
    plot_graph_structure,
    plot_cluster_size_distribution,
)

# Example coordination histogram for chosen central species (if plot_utils available)
try:
    if coord:
        plot_coordination_histograms(coord, central_type=central)
except Exception as e:
    print('Coordination histogram plotting unavailable or failed:', e)

# Graph structure plots
try:
    plot_graph_structure(G, title='Voronoi Graph Structure (full, OVITO)')
    plot_graph_structure(Gm, title='Voronoi Graph Structure (metals-only, OVITO)')
except Exception as e:
    print('Graph plotting unavailable or failed:', e)

# Cluster size distribution from metals-only graph components
try:
    comps = list(nx.connected_components(Gm))
    sizes = [len(c) for c in comps]
    if sizes:
        plot_cluster_size_distribution(sizes, title='Cluster Size Distribution (metals-only, OVITO)')
except Exception as e:
    print('Cluster size distribution plotting unavailable or failed:', e)



In [ ]:
# 3D graph with edges (largest components)
from plot_utils import plot_3d_graph_components

# Full graph (may be dense/noisy); metals-only is typically more informative
try:
    print('3D components for metals-only Voronoi graph:')
    plot_3d_graph_components(Gm, max_components=6)
except Exception as e:
    print('3D graph plotting failed:', e)



In [ ]:
# Average cluster size vs facet area (min_area) sweep using OVITO pipeline
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from voronoi_ovito_utils import (
    build_voronoi_graph_metals_only_from_pipeline,
)

metal_species = ['Pu', 'Na']

# # Collect edge areas at min_area=0 to set sweep range
# G0 = build_voronoi_graph_metals_only_from_pipeline(aae, frame=frame, min_area=0.0, metal_species=metal_species)
# areas0 = np.array([edata.get('area', 0.0) for _,_,edata in G0.edges(data=True)], float)

# if areas0.size == 0:
#     thresholds = np.linspace(0.0, 1.0, 201)
# else:
#     lo = float(np.quantile(areas0, 0.00))
#     hi = float(np.quantile(areas0, 0.80))
#     thresholds = np.linspace(lo, hi if hi > lo else lo + 1.0, 25)
thresholds = [0,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,0.2,0.5,1]

avg_sizes = []
for t in thresholds:
    Gm = build_voronoi_graph_metals_only_from_pipeline(aae, frame=frame, min_area=float(t), metal_species=metal_species)
    comps = list(nx.connected_components(Gm))
    sizes = np.array([len(c) for c in comps], int)
    avg_sizes.append(float(np.mean(sizes)) if sizes.size > 0 else 0.0)

plt.figure(figsize=(6,4))
plt.plot(thresholds, avg_sizes, marker='o')
plt.xlabel('min_area (facet area threshold)')
plt.ylabel('Average cluster size')
plt.title('Average cluster size vs facet area (OVITO metals-only)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
avg_sizes

- Lower conc
- Without Na you do
- last hunder snapshots and avg them
- cluster size as a func of conc
- histogram of CN distribution
- how does avg coordination number change when you change all these parameters.

In [ ]:
- Look at distribution of coordination number
- what did we use as our thesholds